In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as od

In [ ]:
od.download('https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: rajesh22ed
Your Kaggle Key: ··········


100%|██████████| 25.7M/25.7M [00:00<00:00, 113MB/s]


In [ ]:
import pandas as pd
import numpy as np
import gensim

In [ ]:
df = pd.read_csv('/content/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')

In [ ]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
from gensim.utils import simple_preprocess

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
stop_words = stopwords.words('english')
', '.join(stop_words)

"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [ ]:
stop_words = ', '.join(stop_words)[:710]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_text,test_text , train_label, test_label = train_test_split(df.review,df.sentiment,test_size=0.2,random_state=42)

In [ ]:
text = []
for sent in train_text:
  token = simple_preprocess(sent)
  token = [word for word in token if word not in stop_words]
  text.append(token)

In [ ]:
from gensim.models import Word2Vec

In [ ]:
model = Word2Vec(text,vector_size=150,min_count=1)

In [ ]:
model.wv['good'].shape

(150,)

In [ ]:
def avg_wordDoc(doc):
  return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key],axis=0)

In [ ]:
import tqdm
from tqdm import tqdm

In [ ]:
text_ = []
for i in tqdm(range(len(text))):
  text_.append(avg_wordDoc(text[i]))

100%|██████████| 40000/40000 [15:17<00:00, 43.62it/s]


In [ ]:
new_text = np.array(text_)

In [ ]:
new_text.shape

(40000, 150)

In [ ]:
train_embed = new_text

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input,Dense,Dropout,LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential

In [ ]:
train_label = train_label.map({
    'negative':0,
    'positive':1
})

In [ ]:
model = Sequential([
    Input(shape=(150,)),
    Dense(68,activation='relu'),
    Dropout(0.2),
    Dense(32,activation='relu'),
    Dropout(0.2),
    Dense(1,activation='sigmoid')
])

model.compile(optimizer= Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics = ['accuracy'])


In [ ]:
train_label.value_counts()

0    20039
1    19961
Name: sentiment, dtype: int64

In [ ]:
model.fit(train_embed,train_label,epochs=10,validation_split=0.2)

Epoch 1/10
1000/1000 [==============================] - 2s 2ms/step - loss: 0.3321 - accuracy: 0.8607 - val_loss: 0.3152 - val_accuracy: 0.8645
Epoch 2/10
1000/1000 [==============================] - 3s 3ms/step - loss: 0.3271 - accuracy: 0.8624 - val_loss: 0.3233 - val_accuracy: 0.8612
Epoch 3/10
1000/1000 [==============================] - 3s 3ms/step - loss: 0.3245 - accuracy: 0.8623 - val_loss: 0.3161 - val_accuracy: 0.8631
Epoch 4/10
1000/1000 [==============================] - 2s 2ms/step - loss: 0.3221 - accuracy: 0.8638 - val_loss: 0.3122 - val_accuracy: 0.8664
Epoch 5/10
1000/1000 [==============================] - 3s 3ms/step - loss: 0.3199 - accuracy: 0.8656 - val_loss: 0.3105 - val_accuracy: 0.8655
Epoch 6/10
1000/1000 [==============================] - 2s 2ms/step - loss: 0.3157 - accuracy: 0.8661 - val_loss: 0.3170 - val_accuracy: 0.8630
Epoch 7/10
1000/1000 [==============================] - 3s 3ms/step - loss: 0.3149 - accuracy: 0.8678 - val_loss: 0.3184 - val_accuracy:

In [ ]:
sample = train_embed.copy()

In [ ]:
sample = sample.reshape(40000,1,-1)
sample[0].shape

(1, 150)

In [ ]:
model1 = Sequential([
    Input(shape=(1,150)),
    LSTM(68,activation='relu',input_shape=(150,),return_sequences=True),
    Dropout(0.2),
    LSTM(32,activation='relu'),
    Dropout(0.2),
    Dense(1,activation='sigmoid')
])

model1.compile(optimizer= Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics = ['accuracy'])

In [ ]:
model1.fit(sample,train_label,epochs=5,validation_split=0.2)

Epoch 1/5
1000/1000 [==============================] - 9s 6ms/step - loss: 0.3934 - accuracy: 0.8301 - val_loss: 0.3352 - val_accuracy: 0.8556
Epoch 2/5
1000/1000 [==============================] - 5s 5ms/step - loss: 0.3447 - accuracy: 0.8527 - val_loss: 0.3285 - val_accuracy: 0.8604
Epoch 3/5
1000/1000 [==============================] - 5s 5ms/step - loss: 0.3376 - accuracy: 0.8569 - val_loss: 0.3216 - val_accuracy: 0.8605
Epoch 4/5
1000/1000 [==============================] - 6s 6ms/step - loss: 0.3332 - accuracy: 0.8605 - val_loss: 0.3317 - val_accuracy: 0.8570
Epoch 5/5
1000/1000 [==============================] - 4s 4ms/step - loss: 0.3305 - accuracy: 0.8602 - val_loss: 0.3167 - val_accuracy: 0.8620


In [ ]:
testing = "i don't recommend anyone to watch this movie. This movie doesn't fulfill the expectation. Feels like wasted some amount of money"

In [ ]:
token_ = simple_preprocess(testing)
token_ = [word for word in token_ if word not in stop_words]

In [ ]:
testing__ = avg_wordDoc(token_)
test_array = np.array(testing__)

In [ ]:
model1.predict(test_array.reshape(1,1,-1))

1/1 [==============================] - 0s 40ms/step


array([[0.00018998]], dtype=float32)